# LU-разложение и LUP-разложение

<h3>Царитова Нина 

In [2]:
import numpy as np

## LU-разложение

<h0>- это представление этой матрицы в виде произведения A = LU, где L - нижняя треугольная матрица, U - верхняя треугольная ступенчатая матрица.

Функция «LU_decomposition» раскладывает квадратную матрицу А на матрицы L и U 

In [3]:
def LU_decomposition(A):
    n = A.shape[0]
    L = np.identity(n)
    U = A.copy()
    
    for i in range(n):
        multiplier = U[i + 1 :, i] / U[i, i]
        L[i + 1 :, i] += multiplier
        multiplier.resize(multiplier.size, 1)
        U[i + 1 :] -= multiplier * U[i]
        
    return L, U

Задаём квадратную матрицу 3x3

In [4]:
A = np.array([[10, 20, 30.], 
              [0, -50, 60.], 
              [70, 80, 90.]])

Проверяем работу функции

In [34]:
print('L =\n', L)
print('U =\n', U)

L =
 [[1.  0.  0. ]
 [0.  1.  0. ]
 [7.  1.2 1. ]]
U =
 [[  10.   20.   30.]
 [   0.  -50.   60.]
 [   0.    0. -192.]]


Проверяем А = LU

In [33]:
L, U = LU_decomposition(A)
print(A == L.dot(U))

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


Функция «SLAE_LU» находит решение СЛАУ с помощью LU-разложения. Функция принимает матрицу А и вектор-столбец х, возвращает вектор-столбец решений с округлением до сотых

In [7]:
# y = Ux
# b = Ly

def SLAE_LU(A, b): 
    
    L, U = LU_decomposition(A)
    n = L.shape[0]
    y = np.zeros_like(b, dtype=np.double) # Возвращает массив нулей той же формы и типа, что и заданный массив
    y[0] = b[0] / L[0, 0] # y1 = b1/l11
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i] # (bi - sum(from 1 to i-1)(Lij*yj))/Lii

    m = L.shape[0]
    x = np.zeros_like(y, dtype=np.double)
    x[-1] = y[-1] / U[-1, -1] # # ym = bm/lm
    
    for i in range(m-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i] # (yi - sum(from i+1 to m)(Uij*xj))/Uii

    return np.round(x, 2)

Задаём вектор-столбец

In [8]:
b = np.array([[30], 
              [20], 
              [10]])

Проверяем работу функции

In [32]:
print(SLAE_LU(A, b))

[[-2.5 ]
 [ 1.  ]
 [ 1.17]]


Проверяем, совпадает ли результат при использовании linalg.solve

In [31]:
print(np.round(np.linalg.solve(A, b), 2))

[[-2.5 ]
 [ 1.  ]
 [ 1.17]]


Проверяем, совпадает ли результат при использовании lu_factor и lu_solve библиотеки scipy.linalg

In [35]:
from scipy.linalg import lu_factor, lu_solve
print(np.round(lu_solve(lu_factor(A), b), 2))

[[-2.5 ]
 [ 1.  ]
 [ 1.17]]


## LUP-разложение

<h0>- это представление этой матрицы в виде PA = LU, где L - нижняя треугольная матрица, U - верхняя треугольная ступенчатая матрица, P - матрица перестановок.

Функция «swap_rows» меняет местами строки в матрице

In [12]:
def swap_rows(A, a, b):
    t = np.copy(A[a])
    A[a] = A[b]
    A[b] = t

Функция «swap_cols» меняет местами столбцы в матрице

In [13]:
def swap_cols(A, a, b):
    A = np.transpose(A)
    A = swap_rows(A, a, b)
    return np.transpose(A)

Функция «LUP_decomposition» раскладывает квадратную матрицу А на матрицы L (нижняя треугольная), U (верхняя треугольная) и P (матрица перестановок)

In [14]:
def LUP_decomposition(A):
    n = A.shape[0]
    L = np.identity(n)
    U = np.copy(A)
    P = np.identity(n)
    
    for a in range(n):
        b = a + 1
        while U[a][a] == 0:
            swap_rows(U, a, b)
            swap_rows(P, a, b)
            swap_rows(L, a, b)
            swap_cols(L, a, b)
            b += 1
        
        multiplier = U[a + 1 :, a] / U[a, a]
        L[a + 1 :, a] += multiplier
        multiplier.resize(multiplier.size, 1)
        U[a + 1 :] -= multiplier * U[a]
        
    return L, U, P

Проверяем работу функции при тех же входных данных

In [36]:
L, U, P =LUP_decomposition(A)
print('L =\n', L)
print('U =\n', U)
print('P =\n', P)

L =
 [[1.  0.  0. ]
 [0.  1.  0. ]
 [7.  1.2 1. ]]
U =
 [[  10.   20.   30.]
 [   0.  -50.   60.]
 [   0.    0. -192.]]
P =
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


Проверяем А = PLU

In [37]:
L, U, P = LUP_decomposition(A)
print(A == P.dot(L.dot(U)))

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


Проверяем PА = LU

In [38]:
print(P.dot(A) == L.dot(U))

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


Функция «SLAE_LUP» находит решение СЛАУ с помощью LUP-разложения. Функция принимает матрицу А и вектор-столбец х, возвращает вектор-столбец решений с округлением до сотых

In [40]:
def LUP(A, b):
    L, U, P = LUP_decomposition(A)
    b = P.dot(b)
    n = L.shape[0]
    y = np.zeros_like(b, dtype=np.double) # Возвращает массив нулей той же формы и типа, что и заданный массив
    
    y[0] = b[0] / L[0, 0] # y1 = b1/l11
    
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i] # (bi - sum(from 1 to i-1)(Lij*yj))/Lii
    m = L.shape[0]
    x = np.zeros_like(y, dtype=np.double)
    x[-1] = y[-1] / U[-1, -1] # # ym = bm/lm
    
    for i in range(m-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i] # (yi - sum(from i+1 to m)(Uij*xj))/Uii

    return np.round(x, 2)

Проверяем работу функции

In [41]:
print(LUP(A, b))

[[-2.5 ]
 [ 1.  ]
 [ 1.17]]


Проверяем, совпадает ли результат при использовании linalg.solve

In [42]:
print(np.round(np.linalg.solve(A, b), 2))

[[-2.5 ]
 [ 1.  ]
 [ 1.17]]


Проверяем, совпадает ли результат при использовании lu_factor и lu_solve библиотеки scipy.linalg

In [43]:
print(np.round(lu_solve(lu_factor(A), b), 2))

[[-2.5 ]
 [ 1.  ]
 [ 1.17]]


## Сравнение LU и LUP

Поменяем местами строки в матрице так, чтобы на главной диагонали появился 0

In [22]:
swap_rows(A, 0, 1)

Можем заметить, что с помощью LU-разложения мы не можем разложить такую матрицу А на матрицы L и U, поскольку ведущий элемент на главной диагонали матрицы А равен нулю и при приведении матрицы А к ступенчатому виду происходит деление на 0

In [46]:
L, U = LU_decomposition(A)
print('L =\n', L)
print('U =\n', U)

L =
 [[1.  0.  0. ]
 [0.  1.  0. ]
 [7.  1.2 1. ]]
U =
 [[  10.   20.   30.]
 [   0.  -50.   60.]
 [   0.    0. -192.]]


Но LUР-разложене для таких матриц работает

In [45]:
L, U, P =LUP_decomposition(A)
print('L =\n', L)
print('U =\n', U)
print('P =\n', P)

L =
 [[1.  0.  0. ]
 [0.  1.  0. ]
 [7.  1.2 1. ]]
U =
 [[  10.   20.   30.]
 [   0.  -50.   60.]
 [   0.    0. -192.]]
P =
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


Вернём изначальные матрицу и вектор-столбец и проверим время работы функций для нахождения решения СЛАУ с помощью LU-разложения и LUP-разложения

In [25]:
A = np.array([[10, 20, 30.], 
              [0, -50, 60.], 
              [70, 80, 90.]])
x = np.array([[30], 
              [20], 
              [10]])

In [26]:
%timeit SLAE_LU(A, x)

635 us +- 65.6 us per loop (mean +- std. dev. of 7 runs, 1,000 loops each)


In [27]:
%timeit SLAE_LUP(A, x)

681 us +- 36.7 us per loop (mean +- std. dev. of 7 runs, 1,000 loops each)


## Вывод

1. LU-разложение не подходит для решения СЛАУ, в которых приведение к ступенчатому виду матрицы коэффициентов предполагает перестановку строк, так как возможно появление 0 на главной диагонали, а значит разложение матрицы коэффициаентов на нижнюю треугольную матрицу L и верхнюю треугольную матрицу U невозможно. Однако LUP-разложение подходит для решения таких СЛАУ.

2. Решение СЛАУ с помошью LU-разложения занимет меньше времени, чем решение СЛАУ с помощью LUP-разложения

Таким образом, эффективнее использовать LUP-разложение, так как оно более универсально. Но стоит отметить, что решение с помощью LU-разложения вы найдёте быстрее.